# Neural Network implementation
## Group 57

Based on the Neural Network by Michael Neilsen: https://github.com/mnielsen/neural-networks-and-deep-learning

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
import pandas as pd
import os.path

In [2]:
"""
A neural network impelentation which uses stochastic gradient descent for a 
feedforward neural network.
"""
class Network(object):
    """
    The list ``sizes`` contains the number of neurons in the
    respective layers of the network.  For example, if the list
    was [2, 3, 1] then it would be a three-layer network, with the
    first layer containing 2 neurons, the second layer 3 neurons,
    and the third layer 1 neuron. The first layer is the input, last the output
    """
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = []
        self.weights = []
        for y in sizes[1:]:
            self.biases.append(np.random.randn(y,1))
        for x, y in zip(sizes[:-1], sizes[1:]):
            self.weights.append(np.random.randn(y, x))
    """
    a = activation
    b = bias
    w = weight
    sigmoid : see sigmoid function

    Role : Loops through the whole network and updates each neurons activation 
    using the sigmoid function
    """    
    def feedforward(self, a):

        for b, w in zip(self.biases, self.weights):
            a = self.activation(w, a, b)
        return a
    """
    Train the neural network using mini-batch stochastic gradient descent.
    """
    def SGD(self, X_train, Y_train, X_validation, Y_validation, epochs, mini_batch_size, learning_rate, decay):

        training_data = zip(X_train, Y_train)
        validation_data = zip(X_validation, Y_validation)
    
        """Take the training data and make a list out of it"""
        training_data = list(training_data)
        
        """Check if there is data in the test_data"""
        if validation_data:
            validation_data = list(validation_data)
            n_validation_data = len(validation_data)
        
        """
        Mini-batches: Each mini-batch contains mini_batch_size elements from the training set. 
        Splits the training data into mini-batches, and for each mini-batches we train the network. 
        """       
        mini_batches = []
        for j in range(epochs):
            random.shuffle(training_data)
            for k in range(0, len(training_data), mini_batch_size):
                mini_batches.append(training_data[k:k+mini_batch_size])
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, learning_rate)
            if validation_data:
                print("Epoch #" + str(j+1), end = '\t')
                self.evaluate(X_validation, Y_validation)
            else:
                print("Epoch {} complete".format(j))
            learning_rate = learning_rate * (1-decay)
    """
    Update the network's weights and biases by applying
    gradient descent using backpropagation to a single mini batch.
    The ``mini_batch`` is a list of tuples ``(x, y)``.
    """
    def update_mini_batch(self, mini_batch, learning_rate):
        """
        Building an empty networks filled with empty 0 
        """
        mini_batch_bias = []
        mini_batch_weights = []
        for b in self.biases:
            mini_batch_bias.append(np.zeros(b.shape))
        for w in self.weights:
            mini_batch_weights.append(np.zeros(w.shape))
        
        """
        x: Features of the instance
        y: Label of the instance
        eta : Learning rate
        Loops through the samples of the mini-batch, calls backprop on each sample. 
        """
        for x, y in mini_batch:
            
            """
            Returns the gradient of the loss function 
            """
            loss_func_bias, loss_func_weight = self.backprop(x, y)
            """
            Updates the mini-batch bias and mini-batch weight by adding their respective loss function to the
            current mini-batch's network
            """
            mini_batch_bias = [ub+lfb for ub, lfb in zip(mini_batch_bias, loss_func_bias)]
            mini_batch_weights = [uw+lfw for uw, lfw in zip(mini_batch_weights, loss_func_weight)]

        
        """
        Updates each weight with the weights calculated in he minibatch
        """
        self.weights = [ self.update_network_weights(current_weight, learning_rate, mini_batch, mini_batch_weight)
                        for current_weight, mini_batch_weight in zip(self.weights, mini_batch_weights)]

        """
        Updates each weight with the bias calculated in he minibach
        """    
        self.biases = [self.update_network_bias(current_bias, learning_rate, mini_batch, mini_batch_bias)
                       for current_bias, mini_batch_bias in zip(self.biases, mini_batch_bias)]
    """
    Return a tuple (updated_bias, updated_weight) representing the
    gradient for the cost function C_x. 
    """     
    def backprop(self, x, y):
        """
        Building an empty network filled with empty 0's
        """ 
        updated_bias = []
        for b in self.biases:
            updated_bias.append(np.zeros(b.shape))        
        
        updated_weight = []
        for w in self.weights:
            updated_weight.append(np.zeros(w.shape))
        
        """
        FEED FORWARD
        """
        
        """
        Input layer's values
        """
        activation = x
    
        """
        List to store all the activations, initialized with the input layer as the first layer
        """
        activation_list = [x]
        
        """
        List to store all the vectorized output of each layer
        """
        z_list = [] 
        
        """
        z : vectorized output of each layer
        z_list : List of vectorized outputs of each layer
        activation : activation of a layer with sigmoid (sigmoid of the vectorized output)
        activation_list : List of activation of each layer with sigmoid
        
        Run through the network and record the activation with and without sigmoid and save it in  
        z_list(without) and activation_list(with)
        """
        for b, w in zip(self.biases, self.weights):
            z = self.vectorized_output(w, activation, b)
            z_list.append(z)
            activation = self.activation(w, activation, b)
            activation_list.append(activation)         
            
        """
        Now that we ran through the network and calculated the activation we go backward through each layer 
        (from output to input) and update the bias and weight linked to the activations in order to get more 
        accurate results.
        """  
        
        """
        Apply output error formula on the last layer of the activation_list and z_list
        """
        output_layer_error = self.output_layer_error(activation_list, z_list , y)
        
        """
        BACKPROPAGATION
        """
        
        """
        Update bias and weights of the last layer
        """
        updated_bias[-1] = output_layer_error
        updated_weight[-1] = np.dot(output_layer_error, activation_list[-2].transpose())
        
        output_error = output_layer_error
        for l in range(2, self.num_layers):
            updated_bias[-l] = self.output_error(l, updated_bias[-l+1] , z_list)
            updated_weight[-l] = self.cost_weight(l, updated_bias[-l], activation_list)
        return (updated_bias, updated_weight)       
        
    """
    Return the number of test inputs for which the neural
    network outputs the correct result. Note that the neural
    network's output is assumed to be the index of whichever
    neuron in the final layer has the highest activation.
    """   
    def evaluate(self, X_test, Y_test):

        test_data = zip(X_test, Y_test)
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        
        result = (sum(int(x == y) for (x, y) in test_results) / 100)
        print("Accuracy : "+ str(result) + "%")
    
    """
    Mathematical Functions used in the network
    """
    
    def vectorized_output(self, w, a, b):
        return np.dot(w, a)+b
    
    def activation(self, w, a, b):
        return sigmoid(np.dot(w, a)+b)
    
    def output_layer_error(self, activation_list, z_list , y):
        return (activation_list[-1] - y) * sigmoid_prime(z_list[-1])
    
    def output_error(self, l, output_error, z_list):
        return np.dot(self.weights[-l+1].transpose(), output_error) * sigmoid_prime(z_list[-l])

    def cost_weight(self, l, output_error, activation):
        return np.dot(output_error, activation[-l-1].transpose())
    
    def update_network_weights(self, current_weight, learning_rate, mini_batch, mini_batch_weight):
        return current_weight - (learning_rate/len(mini_batch)) * mini_batch_weight
                                                         
    def update_network_bias(self, current_bias, learning_rate, mini_batch, mini_batch_bias):
        return current_bias - (learning_rate/len(mini_batch)) * mini_batch_bias

In [3]:
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [4]:
"""
mnist_loader
~~~~~~~~~~~~
A library to load the MNIST image data.
"""

"""
Return the MNIST data as a tuple containing the training data,
the validation data, and the test data.
"""
def load_data():
    """
    The MNIST data set contains 50k entries, with each image being a
    28*28 pixel array (total_pixels).
    """
    total_pixels = 784
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    
    X_train = [np.reshape(x, (total_pixels, 1)) for x in training_data[0]]
    Y_train = [vectorized_result(y) for y in training_data[1]]
    
    X_validation = [np.reshape(x, (total_pixels, 1)) for x in validation_data[0]]
    Y_validation = validation_data[1]
    
    X_test = [np.reshape(x, (total_pixels, 1)) for x in test_data[0]]
    Y_test = test_data[1]
    
    return (X_train, Y_train, X_validation, Y_validation,  X_test, Y_test)

"""
Return a 10-dimensional unit vector with a 1.0 in the jth
position and zeroes elsewhere.  This is used to convert a digit
(0...9) into a corresponding desired output from the neural
network.
"""
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

# Execute and build

In [6]:
"""
Load the training and validation data, create a Network object and run the SGD algorithm with it.
Accuracy will be printed with the Epoch #.
"""
X_train, Y_train, X_validation, Y_validation,  X_test, Y_test= load_data()

net = Network([784, 30, 10])
net.SGD(X_train, Y_train, X_validation, Y_validation, 10, 10, 2, 0.2)

Epoch #1	Accuracy : 81.48%
Epoch #2	Accuracy : 91.83%


KeyboardInterrupt: 

In [7]:
"""
Evaluate the network.
"""
net.evaluate(X_test, Y_test)

Accuracy : 92.09%


# Testing (random grid search)
For this script to run correctly, the following changes must be made to the Network class (replace current SGD and evaluate functions with below):

In [8]:
def SGD(self, X_train, Y_train, X_validation, Y_validation, epochs, mini_batch_size, learning_rate, decay):
    training_data = zip(X_train, Y_train)
    validation_data = zip(X_validation, Y_validation)
    training_data = list(training_data)
    if validation_data:
        validation_data = list(validation_data)
        n_validation_data = len(validation_data)
#       Updated for the testing
#       ========================
    mini_batches = []
    high_score = [0,0]
    for j in range(epochs):
        random.shuffle(training_data)
        for k in range(0, len(training_data), mini_batch_size):
            mini_batches.append(training_data[k:k+mini_batch_size])
        for mini_batch in mini_batches:
            self.update_mini_batch(mini_batch, learning_rate)
        if validation_data:
            new_score = self.evaluate(X_validation, Y_validation)
            if high_score[0] < new_score:
                high_score[0] = new_score
                high_score[1] = j + 1
        learning_rate = learning_rate * (1-decay)
    return high_score[0], high_score[1]
#       ========================
       
def evaluate(self, X_test, Y_test):
    test_data = zip(X_test, Y_test)
    test_results = [(np.argmax(self.feedforward(x)), y)
                    for (x, y) in test_data]
#       Updated for the testing
#       ========================
    return (sum(int(x == y) for (x, y) in test_results) / 100)
#       ========================

In [9]:
"""
This method tests all combinations inputted manually inside the script to try to replicate
a random grid search method to find an optimum. The values inputted were backed up by 
literature and can be found in the document.
This script writes to an external csv file which doesn't need to exist before calling, and if 
one does exist the values are appended, not overwritten. Stopping mid-script will lose any
accumulated data as the table is only written to on the completion of the algorithm, this can
be modified by putting the df.to_csv methods within the loop - after the df.append function.
"""
def random_grid_search():
    # name the cols -- DON'T CHANGE
    cols = ["# of hidden layers", "# of total neurons", "list of layers", "epoch", "mini-batch size", \
            "learning rate", "learning rate decay", "Winning epoch", "Acc Valid", "Acc Test"]
    # create the data frame
    df = pd.DataFrame(columns=cols)

    ####### Change these values to test different combinations
    epochs_test = [10]
    mini_batch_size_test = np.arange(1, 100, 10).tolist()
    learning_rate_test = np.arrange(0.25, 10, 0.25).tolist()
    learning_rate_decay_test = [0.1, 0.01, 0.001]
    network_shapes = [[784, 30, 5, 10],[784, 30, 10, 10], [784, 30, 30, 10]]
    #######
    
    percentage = 0.0
    X_train, Y_train, X_validation, Y_validation,  X_test, Y_test= load_data()

    for e_idx, e in enumerate(epochs_test):
        for mbs_idx, mbs in enumerate(mini_batch_size_test):
            for lr_idx, lr in enumerate(learning_rate_test):
                for lrd_idx, lrd in enumerate(learning_rate_decay_test):
                    for network_shape in network_shapes:

                        net = Network(network_shape)
                        acc_valid, epoch_count = net.SGD(X_train, Y_train, X_validation, Y_validation, e, mbs, lr, lrd)
                        acc_test = net.evaluate(X_test, Y_test)
                        # append to the df 

                        n_neurons = sum(network_shape) - 794 # 798 is input + ouput layer which can't be changed
                        n_hidden_layers = len(network_shape)-2

                        df = df.append({"# of hidden layers": n_hidden_layers, 
                                        # Change this to lamda loop when we modify the actual shape etc
                                        "# of total neurons": n_neurons,   
                                        "list of layers": network_shape,        
                                        "epoch" : e,                
                                        "mini-batch size" : mbs,    
                                        "learning rate" : lr,       
                                        "learning rate decay" : lrd, 
                                        "Winning epoch" : epoch_count,
                                        "Acc Valid": acc_valid,     
                                        "Acc Test": acc_test},      
                                       ignore_index=True)
                         # sanity check
                        percentage += (1.0 / len(epochs_test) / len(mini_batch_size_test) / len(learning_rate_test) \
                                       / len(learning_rate_decay_test) /len(network_shapes))
                        print("Progress: ", '{0:.2f}'.format(percentage * 100), "%")

    # name your file
    csv_file = 'test_script_results.csv'

    # write to file
    '''If file exists, assume there is a header already'''
    if os.path.isfile(csv_file):
        df.to_csv(csv_file, mode='a', index=False, header=False)
    else:
        df.to_csv(csv_file, mode='a', index=False, header=True)